In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time, timedelta, datetime
from heystox_intraday.trading_indicator import *

In [2]:
raju = ["adanient", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
sorted_stock = SortedStocksList.objects.get(symbol__symbol="spencers", created_at__date=datetime.now().date())

In [3]:
macd_indicator = Indicator.objects.get(name="MACD")
df = get_macd_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df[75:]
if sorted_stock.entry_type in ["SELL", "SS"]:
    last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
elif sorted_stock.entry_type in ["BUY", "SB"]:
    last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
df_after_last_crossover = df.loc[df["date"] >= last_crossover.date]
try:
    if last_crossover.signal == "SELL_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.macd_diff <= -0.070)].iloc[0]
    elif last_crossover.signal == "BUY_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.macd_diff >= 0.070)].iloc[0]
except:
    crossover_signal = None
# df.head(50)
df.loc[(df.macd_diff >= 0.070)]
last_crossover
crossover_signal

IndexError: single positional indexer is out-of-bounds

In [ ]:
macd_indicator = Indicator.objects.get(name="MACD")
df = get_stochastic_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df[75:]
if sorted_stock.entry_type in ["SELL", "SS"]:
    last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
elif sorted_stock.entry_type in ["BUY", "SB"]:
    last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
df_after_last_crossover = df.loc[df["date"] >= last_crossover.date]
try:
    if last_crossover.signal == "SELL_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.stoch_diff <= -22.70)].iloc[0]
    elif last_crossover.signal == "BUY_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.stoch_diff >= 22.70)].iloc[0]
except:
    crossover_signal = None
df_after_last_crossover

In [ ]:
raju = ["adanient", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
# fifth = date(2020,1,25)
# for r in raju:
#     symbol = Symbol.objects.get(symbol=r)
#     SortedStocksList.objects.create(symbol=symbol, entry_type="BUY")
# # Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju)
# for candle in Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju):
#     candle.date = candle.date + timedelta(1)
#     candle.save()

In [ ]:
from heystox_intraday.intraday_functions_strategy import get_stochastic_crossover, get_macd_crossover

In [4]:
liquid_stocks = [SortedStocksList.objects.get_or_create(symbol=Symbol.objects.get(symbol=r), entry_type="SS") for r in raju]
liquid_stocks

[(<SortedStocksList: adanient | 2020-01-28>, False),
 (<SortedStocksList: bhel | 2020-01-28>, False),
 (<SortedStocksList: dlf | 2020-01-28>, False),
 (<SortedStocksList: gmrinfra | 2020-01-28>, False),
 (<SortedStocksList: gail | 2020-01-28>, False),
 (<SortedStocksList: infratel | 2020-01-28>, False),
 (<SortedStocksList: ioc | 2020-01-28>, False),
 (<SortedStocksList: idea | 2020-01-28>, False),
 (<SortedStocksList: nhpc | 2020-01-28>, False),
 (<SortedStocksList: pnb | 2020-01-28>, False),
 (<SortedStocksList: relinfra | 2020-01-28>, False),
 (<SortedStocksList: recltd | 2020-01-28>, False),
 (<SortedStocksList: spencers | 2020-01-28>, False),
 (<SortedStocksList: tatapower | 2020-01-28>, False),
 (<SortedStocksList: vakrangee | 2020-01-28>, False),
 (<SortedStocksList: yesbank | 2020-01-28>, False),
 (<SortedStocksList: ujjivansfb | 2020-01-28>, False),
 (<SortedStocksList: granules | 2020-01-28>, False),
 (<SortedStocksList: mothersumi | 2020-01-28>, False),
 (<SortedStocksList: 